In [41]:
import numpy as np 
import pandas as pd
import random
import datetime
import os
from math import sin, cos, sqrt, atan2, radians

def oneHotForFeatures(dfDriverOrders, columnName):
    onehot = pd.get_dummies(dfDriverOrders[columnName],prefix=columnName)
    dfDriverOrders = dfDriverOrders.drop(columnName, axis =1)
    dfDriverOrders = dfDriverOrders.join(onehot)
    return dfDriverOrders
    
def calculateDuration(driverRow):
    meansOfTransport = (int)(driverRow['meansOfTransport'])
    distance = (float)(driverRow['distance'])
    duration = (float)(distance / (70 * (1 + meansOfTransport /10)))

    return duration * 60

def calculateCost(driverInfo, driverOrderRow):
    basesalary = driverInfo['basesalary']
    costPerKM = driverInfo['perkm']
    
    duration = driverOrderRow['duration']
    distance = driverOrderRow['distance']
    
    delay = driverOrderRow['delay']
    trafficviolations = driverOrderRow['trafficViolation']
    
    return basesalary * duration + costPerKM * distance + delay * costPerKM 
    + trafficviolations * 200 + random.randint(0, basesalary * costPerKM)

def getDriverInfoRow(driverName, dataFrameDrivers):
    for index in range(0,len(dataFrameDrivers)-1):
        eachRow = dataFrameDrivers.iloc[index]
        if (eachRow['driver'] == driverName):
             return eachRow
    return 20

currentpath = './'
dataFrameDrivers = pd.read_csv(currentpath + "drivers.csv")
#dataFrameDrivers = dataFrameDrivers.set_index('driver')
finalPath = 'final/'

outputpath = 'output/'
listingDriverFiles = os.listdir(outputpath)
for infile in listingDriverFiles:
    print("current file is: " + infile)
    dfDriverOrders = pd.read_csv(outputpath + infile)
    dfDriverOrders.drop(dfDriverOrders.columns[[0]], axis=1, inplace=True)
    dfDriverOrders = dfDriverOrders.drop('startTime', 1)
    dfDriverOrders = dfDriverOrders.drop('endTime', 1)
    
    dfDriverOrders = oneHotForFeatures(dfDriverOrders, 'startLocation')
    dfDriverOrders = oneHotForFeatures(dfDriverOrders, 'endLocation')
    dfDriverOrders = oneHotForFeatures(dfDriverOrders, 'timespan')
    
    #Get driver info
    driverName = infile.split('.')[0]
    print("driverName", driverName)
    driverInfo = getDriverInfoRow(driverName, dataFrameDrivers)
    
    #calculate duration and cost
    for rowIndex in range(0,len(dfDriverOrders)):
        dfDriverOrders.set_value(rowIndex, 'duration', calculateDuration(dfDriverOrders.iloc[rowIndex])) 
        dfDriverOrders.set_value(rowIndex, 'cost', calculateCost(driverInfo,dfDriverOrders.iloc[rowIndex]))
    
    dfDriverOrders = dfDriverOrders.drop('driver', 1)
    
    cols = dfDriverOrders.columns.tolist()
    cols.insert(len(cols), cols.pop(5))

    dfDriverOrders = dfDriverOrders[cols]     
    dfDriverOrders.to_csv(finalPath + infile)
    
    






current file is: Bang.csv
driverName Bang
current file is: Christy.csv
driverName Christy
current file is: Kenneth.csv
driverName Kenneth
current file is: Kristin.csv
driverName Kristin
current file is: Ling.csv
driverName Ling
current file is: Lingqi.csv
driverName Lingqi
current file is: Patric.csv
driverName Patric
current file is: Robin.csv
driverName Robin
current file is: Sage.csv
driverName Sage
current file is: Sue.csv
driverName Sue
current file is: Summer.csv
driverName Summer
current file is: Zion.csv
driverName Zion
